# 集团成绩分析

清空数据库
``` python
delete = db.finalexam.delete_many({})
delete.deleted_count
```

In [1]:
import xlwings as xw
from pymongo import MongoClient, errors
from cytoolz import itertoolz

In [2]:
client = MongoClient('mongodb://10.57.0.92:27017')
db = client['maple_pds']
try:
    Cs = db.collection_names()
except errors.ServerSelectionTimeoutError:
    print('failed!')
else:
    print(db.collection_names())

['finalexam', 'users', 'examinfos', 'staffinfos', 'studentinfos', 'gradeinfos', 'courseinfos']


In [28]:
result = db.finalexam.aggregate([
        {"$match":{ "score.math.state": 0}},
        {'$group' : {
            '_id' : {"grade_id": "$grade_id"},
            'avg' : {'$avg' : "$score.math.final"},
            'total': {'$sum' : 1},
            's':{'$sum': {'$cond': [{"$lt":['$score.math.final', {'$multiply':['$score.math.full',0.6]}]}, 1, 0]}}
        }
    }])
list(result)

[{'_id': {'grade_id': 8}, 'avg': 68.42105263157895, 's': 11, 'total': 19},
 {'_id': {'grade_id': 7}, 'avg': 64.91891891891892, 's': 22, 'total': 37},
 {'_id': {'grade_id': 6}, 'avg': 82.35185185185185, 's': 2, 'total': 27},
 {'_id': {'grade_id': 5}, 'avg': 81.43478260869566, 's': 1, 'total': 23},
 {'_id': {'grade_id': 4}, 'avg': 84.62068965517241, 's': 0, 'total': 29},
 {'_id': {'grade_id': 3}, 'avg': 82.28947368421052, 's': 0, 'total': 38},
 {'_id': {'grade_id': 2}, 'avg': 88.4322033898305, 's': 2, 'total': 59},
 {'_id': {'grade_id': 1}, 'avg': 93.0, 's': 2, 'total': 70}]

In [42]:
result = db.finalexam.aggregate([
    {"$match": {"score.math.state": 0}},
    {'$group': {
        '_id': {"grade_id": "$grade_id"},
        'avg': {'$avg': "$score.math.final"},
        'total': {'$sum': 1},
        's1': {'$sum': {'$cond': [{
            '$and': [
                {"$lt": ['$score.math.final', {
                        '$multiply': ['$score.math.full', 0.6]}]},
                {"$gte": ['$score.math.final', {
                    '$multiply': ['$score.math.full', 0]}]}
            ]
        }, 1, 0]}},
        's2': {'$sum': {'$cond': [{
            '$and': [
                        {"$lt": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.8]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.6]}]}
                        ]
        }, 1, 0]}},
        's3': {'$sum': {'$cond': [{
            '$and': [
                        {"$lt": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.9]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.8]}]}
                        ]
        }, 1, 0]}},
        's4': {'$sum': {'$cond': [{
            '$and': [
                        {"$lte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 1.0]}]},
                        {"$gte": ['$score.math.final', {
                            '$multiply': ['$score.math.full', 0.9]}]}
                        ]
        }, 1, 0]}},
    }
    }])
list(result)

[{'_id': {'grade_id': 8},
  'avg': 68.42105263157895,
  's1': 11,
  's2': 5,
  's3': 3,
  's4': 0,
  'total': 19},
 {'_id': {'grade_id': 7},
  'avg': 64.91891891891892,
  's1': 22,
  's2': 12,
  's3': 2,
  's4': 1,
  'total': 37},
 {'_id': {'grade_id': 6},
  'avg': 82.35185185185185,
  's1': 2,
  's2': 2,
  's3': 14,
  's4': 9,
  'total': 27},
 {'_id': {'grade_id': 5},
  'avg': 81.43478260869566,
  's1': 1,
  's2': 6,
  's3': 10,
  's4': 6,
  'total': 23},
 {'_id': {'grade_id': 4},
  'avg': 84.62068965517241,
  's1': 0,
  's2': 7,
  's3': 13,
  's4': 9,
  'total': 29},
 {'_id': {'grade_id': 3},
  'avg': 82.28947368421052,
  's1': 0,
  's2': 14,
  's3': 12,
  's4': 12,
  'total': 38},
 {'_id': {'grade_id': 2},
  'avg': 88.4322033898305,
  's1': 2,
  's2': 4,
  's3': 13,
  's4': 40,
  'total': 59},
 {'_id': {'grade_id': 1},
  'avg': 93.0,
  's1': 2,
  's2': 2,
  's3': 8,
  's4': 58,
  'total': 70}]

In [13]:
result = db.finalexam.aggregate([{'$group' : {
            '_id' : {"grade_id": "$grade_id", "score.math.state": "$score.math.state"},
            'avg' : {'$avg' : "$score.math.final"},
            'total': {'$sum' : 1}
        }
    }])

In [14]:
list(result)

[{'_id': {'grade_id': 8, 'score.math.state': 1}, 'avg': None, 'total': 1},
 {'_id': {'grade_id': 1, 'score.math.state': 0}, 'avg': 93.0, 'total': 70},
 {'_id': {'grade_id': 1, 'score.math.state': 1}, 'avg': None, 'total': 2},
 {'_id': {'grade_id': 2, 'score.math.state': 0},
  'avg': 88.4322033898305,
  'total': 59},
 {'_id': {'grade_id': 3, 'score.math.state': 0},
  'avg': 82.28947368421052,
  'total': 38},
 {'_id': {'grade_id': 4, 'score.math.state': 0},
  'avg': 84.62068965517241,
  'total': 29},
 {'_id': {'grade_id': 5, 'score.math.state': 0},
  'avg': 81.43478260869566,
  'total': 23},
 {'_id': {'grade_id': 6, 'score.math.state': 0},
  'avg': 82.35185185185185,
  'total': 27},
 {'_id': {'grade_id': 7, 'score.math.state': 0},
  'avg': 64.91891891891892,
  'total': 37},
 {'_id': {'grade_id': 8, 'score.math.state': 0},
  'avg': 68.42105263157895,
  'total': 19}]

In [3]:
stagings = list(itertoolz.sliding_window(2, [0, 60, 80, 90, 100]))
print(stagings)

[(0, 60), (60, 80), (80, 90), (90, 100)]


In [6]:
score_staging = []
for i in range(1,10):
    cur = db.finalexam.find({'score.math.state':1,'grade_id':i})
    grade = [cur.count()]
    for m, n in stagings:
        # m ≤ score < n
        if i>= 7:
            m, n = m*1.2, n*1.2
        rng = {'$gte': m, '$lt': n}
        cur = db.finalexam.find({'score.math.final':rng, 'score.math.state':0, 'grade_id':i},{'_id': 0})
        grade.append(cur.count())
#         print('%s-%s:\t%s'% (m,n,cur.count()))

    print(grade)
    score_staging.append(reversed(grade))

[2, 2, 2, 8, 51]
[0, 2, 4, 13, 39]
[0, 0, 14, 12, 11]
[0, 0, 7, 13, 9]
[0, 1, 6, 10, 6]
[0, 2, 2, 14, 9]
[0, 22, 12, 2, 1]
[1, 11, 5, 3, 0]
[0, 0, 0, 0, 0]


In [36]:

for m, n in itertoolz.sliding_window(2, staging):
    # m ≤ score < n
    rng = {'$gte': m, '$lt': n}
    cur = db.finalexam.find({'score.math.final':rng,'grade_id':1},{'_id': 0})
    print('%s-%s:\t%s'% (m,n,cur.count()))

0-60:	2
60-80:	2
80-90:	8
90-100:	51


In [52]:
cur = db.finalexam.find({'score.math.state':1,'grade_id':1})
cur.count()

2

In [58]:
score_staging

[[2, 2, 8, 51, 2],
 [2, 4, 13, 39, 0],
 [0, 14, 12, 11, 0],
 [0, 7, 13, 9, 0],
 [1, 6, 10, 6, 0],
 [2, 2, 14, 9, 0],
 [22, 12, 2, 1, 0],
 [11, 5, 3, 0, 1],
 [0, 0, 0, 0, 0]]

In [32]:
cur = db.finalexam.find({'grade_id':1},{'_id': 0})

In [33]:
list(filter(lambda x: x['score']['math']['state'] == 1 ,cur))

[{'class_id': '201501',
  'exam_id': '2016060001',
  'grade_id': 1,
  'name': '刘依诺',
  'score': {'chinese': {'final': '请假', 'full': 100, 'state': 1},
   'math': {'final': '请假', 'full': 100, 'state': 1}},
  'student_id': 'P1500233'},
 {'class_id': '201503',
  'exam_id': '2016060001',
  'grade_id': 1,
  'name': '马子玉',
  'score': {'chinese': {'final': '缺考', 'full': 100, 'state': 1},
   'math': {'final': '缺考', 'full': 100, 'state': 1}},
  'student_id': 'P1500237'}]

In [27]:
dir(cur)

['_Cursor__address',
 '_Cursor__batch_size',
 '_Cursor__check_okay_to_chain',
 '_Cursor__codec_options',
 '_Cursor__collection',
 '_Cursor__comment',
 '_Cursor__data',
 '_Cursor__die',
 '_Cursor__empty',
 '_Cursor__exhaust',
 '_Cursor__exhaust_mgr',
 '_Cursor__explain',
 '_Cursor__hint',
 '_Cursor__id',
 '_Cursor__killed',
 '_Cursor__limit',
 '_Cursor__manipulate',
 '_Cursor__max',
 '_Cursor__max_await_time_ms',
 '_Cursor__max_scan',
 '_Cursor__max_time_ms',
 '_Cursor__min',
 '_Cursor__modifiers',
 '_Cursor__ordering',
 '_Cursor__projection',
 '_Cursor__query_flags',
 '_Cursor__query_spec',
 '_Cursor__read_concern',
 '_Cursor__read_preference',
 '_Cursor__retrieved',
 '_Cursor__send_message',
 '_Cursor__skip',
 '_Cursor__spec',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '

In [15]:
cur.count()

72

In [35]:
def group_score(doc):
    for m, n in itertoolz.sliding_window(2, staging):
        # m ≤ score < n
        rng = {'$gte': m, '$lt': n}
        doc['score']['math']['final']
        cur = db.finalexam.find({'score.math.final':rng,'grade_id':1},{'_id': 0})
        print('%s-%s:\t%s'% (m,n,cur.count()))

In [28]:
cur = db.finalexam.find({'score.math.0':{'$gte': 80, '$lte': 89},'grade_id':1},{'_id': 0})

In [29]:
for row in cur:
    print(row)

{'exam_id': '2016201001', 'score': {'math': [85.5, 100], 'chinese': [93.0, 100]}, 'name': '李嘉耀', 'class_id': '201501', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [87.5, 100], 'chinese': [82.0, 100]}, 'name': '王锦轩', 'class_id': '201502', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [88.5, 100], 'chinese': [90.5, 100]}, 'name': '王馨悦', 'class_id': '201502', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [82.0, 100], 'chinese': [91.5, 100]}, 'name': '赵航标', 'class_id': '201502', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [87.5, 100], 'chinese': [84.0, 100]}, 'name': '吴金辉', 'class_id': '201503', 'grade_id': 1}
{'exam_id': '2016201001', 'score': {'math': [87.0, 100], 'chinese': [84.5, 100]}, 'name': '赵启志', 'class_id': '201503', 'grade_id': 1}
